# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read the .csv fle in WeatherPy
city_weather_info = pd.read_csv("output_data/cities.csv")
city_weather_info

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Alibag,18.6411,72.8792,82.40,78,20,8.90,IN,1619129798
1,Barrow,71.2906,-156.7887,19.40,79,1,8.05,US,1619129324
2,Sørland,67.6670,12.6934,30.20,93,75,3.44,NO,1619129799
3,Norman Wells,65.2820,-126.8329,26.60,50,75,12.66,CA,1619129376
4,Pitimbu,-7.4706,-34.8086,78.80,100,0,6.91,BR,1619129799
...,...,...,...,...,...,...,...,...,...
565,Kamina,-8.7386,24.9906,64.58,95,98,2.84,CD,1619130005
566,Murgab,37.4966,61.9714,55.40,50,5,6.91,TM,1619130005
567,Vao,-22.6667,167.4833,74.08,74,22,6.06,NC,1619130006
568,Gupitan,7.7758,125.6406,73.29,92,39,2.66,PH,1619130006


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
!jupyter nbextension enable --py gmaps

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# find out the center of map:
lat_m = city_weather_info.Lat.mean()
lng_m = city_weather_info.Lng.mean()
print(lat_m,lng_m)

22.051880701754385 20.85533210526316


In [6]:
# Create a Humidity Heatmap layer to the map

locations = city_weather_info[["Lat","Lng"]]

heatmap_layer = gmaps.heatmap_layer(locations, weights = city_weather_info["Humidity"], 
                                    dissipating = False, max_intensity = 100 , point_radius = 3)
fig = gmaps.figure(center = (22,21), zoom_level = 2)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
ndown_cities = city_weather_info.loc[(city_weather_info["Max Temp"] > 70) & (city_weather_info["Max Temp"] < 80) 
                                     & (city_weather_info["Wind Speed"] <10) & (city_weather_info["Cloudiness"] == 0)].dropna()
ndown_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,Pitimbu,-7.4706,-34.8086,78.8,100,0,6.91,BR,1619129799
21,Hong Kong,22.2855,114.1577,77.0,83,0,5.06,HK,1619129222
296,Maceió,-9.6658,-35.7353,77.0,94,0,5.75,BR,1619129780
348,Esmeraldas,-19.7625,-44.3139,75.2,43,0,4.61,BR,1619129919
366,Cabedelo,-6.9811,-34.8339,78.8,100,0,6.91,BR,1619129824
430,Gerāsh,27.6652,54.1371,77.0,22,0,2.71,IR,1619129952
546,Barkhan,29.8977,69.5256,70.9,27,0,3.62,PK,1619129997


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Store into variable named hotel_df
hotel_df = ndown_cities.loc[:,["City","Country","Lat","Lng"]]

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
4,Pitimbu,BR,-7.4706,-34.8086,
21,Hong Kong,HK,22.2855,114.1577,
296,Maceió,BR,-9.6658,-35.7353,
348,Esmeraldas,BR,-19.7625,-44.3139,
366,Cabedelo,BR,-6.9811,-34.8339,
430,Gerāsh,IR,27.6652,54.1371,
546,Barkhan,PK,29.8977,69.5256,


In [16]:
from pprint import pprint
lat_h = hotel_df.iloc[1, 2]
lng_h = hotel_df.iloc[1, 3]
loc_h = f"{lat_h},{lng_h}"

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?radius=1500&type=hotel&keyword=hotel&key={g_key}&location="
response = requests.get(url+loc_h).json()
pprint(response)

{'error_message': 'The provided API key is invalid.',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}


In [ ]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame

for i in range(hotel_df.City.count()):
    lat_h = hotel_df.iloc[i, 2]
    lng_h = hotel_df.iloc[i, 3]
    loc_h = f"{lat_h},{lng_h}"

    response = requests.get(url+loc_h).json()
    results = response["results"]

    try:
        hotel_df.iloc[i, 4] = results[0]["name"]
        city_h = list(hotel_df["City"])
        #print(f"Near {city_h[i]}, hotel name of '{results[0]["name"]}' is found!")
    except:
        print("-----------No results-----------")

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))